# Mask R-CNN Species Measuring

### Environment Setup

In [2]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imgaug
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.whales import whale

import numpy.ma as ma
import scipy.misc

import numpy as np
import matplotlib.pyplot as plt
import skimage.filters

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


### Model and dataset setup

In [3]:
#sets up the system and generates masks
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
%matplotlib inline 

# Path to trained weights
#WHALE_WEIGHTS_PATH = "/home/clifgray/Code/Mask_RCNN/logs/whale20180613T0247/mask_rcnn_whale_0075.h5"

config = whale.WhaleConfig()
WHALE_DIR = os.path.join(ROOT_DIR, "datasets/whale_training/blue")
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
# Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

# Load validation dataset
dataset = whale.WhaleDataset()
dataset.load_whale(WHALE_DIR, "test")
#
# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))


#Getting manually drawn masks
id_array = np.arange(len(dataset.image_ids)) #17 images in this set
mask_array = []
for num in id_array:
    image_id = num

    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
   # if image_id >= 19 and image_id <= 35:
    mask_array.append(mask)
    
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax


Images: 17
Classes: ['BG', 'megaptera_novaeangliae_body', 'megaptera_novaeangliae_pectoral']


## Function Definitions

In [4]:
def run_cnn(weights_path="/home/clifgray/Code/Mask_RCNN/logs/whale20180613T0247/mask_rcnn_whale_0150.h5"):
    # Create model in inference mode
    with tf.device(DEVICE):
        model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)
        
    # Load weights
    print("Loading weights ", weights_path)
    model.load_weights(weights_path, by_name=True)
    print("Weights loaded.")
    
    
    detection_results = []
    initial_time = time.perf_counter()
    for image_id in dataset.image_ids:
    
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

        info = dataset.image_info[image_id]

        # Run object detection
        results = model.detect([image], verbose=0)
        
        detection_results.append([results, info, image_id])
        print("Done detecting and masking image #:  " + str(len(detection_results)))
    
    finish_time = time.perf_counter()
    time_elapsed = finish_time - initial_time
    print("\n")
    print(time_elapsed, "seconds elapsed while masking", len(detection_results), "images.")
    print((time_elapsed/len(detection_results), "seconds per image."))
    
    return detection_results



In [5]:
#Each image could have multiple masks (body and pectoral)
#Only done with the manually drawn masks

def find_correct_mask(mask): 
    #Find which mask to use - one with the largest number of pixels
    # TODO change this to choosing the ones with the body label
    numOfMasks = np.size(mask, 2)
    i = 0
    pixcount = 0
    pixCountArray = []
    while i < numOfMasks:
        currentMask = (mask[:, :, i])
        pixcount = 0
        nonzero = np.nonzero(currentMask)
        it2 = np.nditer(currentMask, flags =['multi_index'])
        while not it2.finished:
            if it2[0] == True:
                pixcount = pixcount + 1
            it2.iternext()
        pixCountArray.append(pixcount)
        i = i+1

#Find max in pixCountArray - gets the biggest mask that will be the body
    correctIndex = pixCountArray.index(max(pixCountArray))
    body = (mask[:,:,correctIndex])
    return body

In [6]:
def pca(body, boolean):
    xy_array = []
 # get indexes of mask pixels
    y, x = np.nonzero(body)
    
# mean center the coords
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])

# build covariance matreix and eigenvectors
    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)
    
# sort eigenvalues
    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]

    
    if boolean == True:
# plot the major and minor axis of the whale mask
        scale = 20
        plt.plot(x, y, 'k.')

        plt.plot([x_v1*-scale*2, x_v1*scale*2],
         [y_v1*-scale*2, y_v1*scale*2], color='red')
        plt.plot([x_v2*-scale, x_v2*scale],
         [y_v2*-scale, y_v2*scale], color='blue')
        plt.axis('equal')
        plt.gca().invert_yaxis()  # Match the image system with origin at top left
        plt.show()

# orient this along the horizontal axis
    theta = np.tanh((x_v2)/(y_v2))  
    rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
                      [np.sin(theta), np.cos(theta)]])
    transformed_mat = rotation_mat * coords
# plot the transformed blob
#these are the final transformed coords
    x_transformed, y_transformed = transformed_mat.A

    maxX = np.max(x_transformed)
    minX = np.min(x_transformed)
    maxY = np.max(y_transformed)
    minY = np.min(y_transformed)


#Get corresonding Y values for minX and maxX
    maxX_index = np.where(x_transformed == maxX) #index of right-most point
    rightY = float((y_transformed[maxX_index])) #corresponding Y value


    minX_index = np.where(x_transformed == minX) #index of left-most point
    leftY = float((y_transformed[minX_index])) #corresponding Y value
    
#Orient the mask correctly - flip so the fluke is on the right

#Get corresonding X values for maxY and minY

    maxY_index = np.where(y_transformed == maxY) #index of top point
    topX = float((x_transformed[maxY_index])) #corresponding X value


    minY_index = np.where(y_transformed == minY) #index of bottom point
    bottomX = float((x_transformed[minY_index])) #corresponding X value

    #Flip mask so fluke is on the right, if necessary
    if (topX < 0 or bottomX < 0):
        x_transformed = x_transformed*-1 
    
    xy_array = [x_transformed, y_transformed]
    
    
    return xy_array


In [7]:
def measure(results_list):
    manual_length_list = []

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    
    #Measuring process is repeated for each item in the array results_list
    for item in results_list:
        if np.array_equal(results_list, detection_results):
            mask = item[0][0]["masks"]
        else:
            mask = item
            
        body = find_correct_mask(mask) #Ignore pectoral masks
        xy_array = pca(body, False) #PCA, boolean argument determines if the images are shown during measuring
        x_transformed = xy_array[0]
        y_transformed = xy_array[1]
        
        #Reassign max/min X values in case image was flipped during PCA

        maxX = np.max(x_transformed) #Right-most point
        minX = np.min(x_transformed) #Left-most point

#Get corresonding Y values for maxX and minX

        maxX_index = np.where(x_transformed == maxX) #index of right-most point
        rightY = float((y_transformed[maxX_index])) #corresponding Y value


        minX_index = np.where(x_transformed == minX) #index of left-most point
        leftY = float((y_transformed[minX_index])) #corresponding Y value
        
#Draw a straight line across the mask

#Filter out points close to the midline of the mask (on the Y axis)
    #Arbitrary lambda, might need to change later
        lowEnough = list(filter(lambda y: y < (leftY + 0.5), y_transformed)) #above midline
        yValues = list(filter(lambda y: y > (leftY - 0.5), lowEnough)) #below midline
        yValues.sort()

#Get corresponding X values to draw the line

#List of appropriate indices
        indices = []
        for point in yValues:
            index = int(np.where(y_transformed == point)[0])
            indices.append(index)
    
        xValues = [] #Corresponding X values
        for index in indices:
            xValues.append(x_transformed[index]) 

        xValues.sort()

        #Use distance formula to measure the length from the midline
        length = math.sqrt((xValues[-1] - xValues[0])**2 + (yValues[-1] - yValues[0])**2)
        
       
        if np.array_equal(results_list, detection_results): # Append to CNN object for auto-generated masks
            item.append(length)
        else:
            manual_length_list.append(length) #Create and return a new list of lengths for manual masks
        
    
    if np.array_equal(results_list, detection_results): # Append to CNN object
        return
    else:
        return manual_length_list
    

In [8]:
def show_example(whale_list, index): # Repeats and displays measuring without saving information, same as measure function
    
    image_id = index
    
    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
        dataset, config, image_id, use_mini_mask=False)
    log("molded_image", image)
    log("mask", mask)
    print("mask.shape", mask.shape)
    
    item = whale_list[index]
    if np.array_equal(whale_list, detection_results):
        mask = item[0][0]["masks"]
    else:
        mask = item 

    visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names, show_bbox=False)
    

    length_list = []

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    
    body = find_correct_mask(mask)
    xy_array = pca(body, True)
    x_transformed = xy_array[0]
    y_transformed = xy_array[1]
        

    maxX = np.max(x_transformed)
    minX = np.min(x_transformed) 
    


    maxX_index = np.where(x_transformed == maxX) 
    rightY = float((y_transformed[maxX_index])) 


    minX_index = np.where(x_transformed == minX) 
    leftY = float((y_transformed[minX_index])) 
        
        
#Get points from leftX and rightX until it runs off the mask

#Filter out points above straight horizontal midline 
    lowEnough = list(filter(lambda y: y < (leftY + 0.5), y_transformed))
    yValues = list(filter(lambda y: y > (leftY - 0.5), lowEnough)) 
    yValues.sort()

#Get corresponding X values
    indices = []
    for point in yValues:
        index = int(np.where(y_transformed == point)[0])
        indices.append(index)
    
    xValues = [] #Corresponding X values
    for index in indices:
        xValues.append(x_transformed[index]) 

    xValues.sort()

    length = math.sqrt((xValues[-1] - xValues[0])**2 + (yValues[-1] - yValues[0])**2)


    #Plot the midline used for measuring
    plt.plot(x_transformed, y_transformed, 'g.', zorder=0)
    
    # set axis limits
    plt.xlim([minX - 100, maxX + 100])
    plt.ylim([leftY - 200, rightY + 200])
    
    #Plot the first and last points from the list, use this for length
    
    plt.scatter(xValues[0], yValues[0], zorder=10)
    plt.scatter(xValues[-1], yValues[-1], zorder=10)
    plt.scatter(xValues, yValues, zorder=10)
    plt.show()
    
    print("Length in Pixels:", length)
    
    


In [9]:
def convert_measurements(results_list):
    excel_measurements = pd.read_excel("NSF-IOS_2017Measurements_kcb_updated_062018.xlsx")
    # Known measurements (imported from excel)
    blue_measurements = excel_measurements.loc[excel_measurements["Whale"].str.contains("Bm")] 
    blue_measurements = blue_measurements.reset_index(drop=True)
    manual_lengths = []
    cnn_lengths = []

# imaged were downsized from 6000 as a max dimension to 1024 as a max dimension so pixels are 5x as large in meters
    pixel_size_factor = 6000.0 / 1024.0

#measurements_df = pd.DataFrame(np.nan, index=np.arange(0, len(blue_measurements)), columns=['Filename', 'Auto_CNN_Measurements', 'Manual_Mask_Measurements', 'Manual_Standard_Measurements', 'Estimated_Manual_Error'])

#measurements_df.set_index("Filename")

    for index, row in blue_measurements.iterrows():
        cnn_pixel_count = 0
        manual_pixel_count = 0
    
        total_altitude = row["Total Altitude"]
        pixel_size = row["Pixel size"]
        focal_length = row["Focal length (mm)"] #always 5
    
        if np.array_equal(results_list, detection_results): 
            for whale in detection_results: #CNN Masks
                if whale[1]['id'] == (row['Image']):
                    cnn_pixel_count = whale[3]
                    adjusted_cnn_pixel_count = cnn_pixel_count * pixel_size_factor
                    cnn_length = adjusted_cnn_pixel_count * pixel_size/focal_length * total_altitude
                    cnn_lengths.append(cnn_length)
       
        else: #Manual Masks
            adjusted_manual_pixel_count = results_list[index] * pixel_size_factor
            manual_length = adjusted_manual_pixel_count * pixel_size/focal_length * total_altitude
            manual_lengths.append(manual_length)
        
    if np.array_equal(results_list, detection_results): 
        return cnn_lengths
    else:
        return manual_lengths

    # TODO need to make this more robust, might not match the correct index at some point

In [10]:
def girth(results_list, length_list, num_of_increments):
    
    girth_measurements = []

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    
    for item in results_list: #get the mask and length
        if np.array_equal(results_list, detection_results):
            mask = item[0][0]["masks"]
            length = item[3]
            img_id = item[2]
        else:
            mask = item
            index = results_list.index(item)
            length = length_list[index]
            img_id = index
            
        body = find_correct_mask(mask) #Ignore pectoral masks
        increment = length / num_of_increments
        
        xy_array = pca(body, False) #PCA
        x_transformed = xy_array[0]
        y_transformed = xy_array[1]
        
        print("Increment in Pixels", increment)
        
        maxX = np.max(x_transformed)
        minX = np.min(x_transformed)
        maxY = np.max(y_transformed)
        minY = np.min(y_transformed)
        
        plt.plot(x_transformed, y_transformed, 'g.', zorder=0)
        
        avgY = (sum(y_transformed) / len(y_transformed))
        currentX = minX
        
        current_whale_girth = []
        while(currentX < maxX-increment): 
            plt.scatter(currentX, avgY)
            
            
            #At each X, filter out Y values from y_transformed to get vertical line
                #Similar to measure()
            #Filter out points close to vertical line 

            right_x_values = list(filter(lambda x: x < (currentX + 0.5), x_transformed))
            xValues = list(filter(lambda x: x > (currentX - 0.5), right_x_values)) 
            xValues.sort()

        #Get corresponding Y values to find top and bottom of line
    #List of appropriate indices
            indices = []
            for point in xValues:
                index = int(np.where(x_transformed == point)[0])
                indices.append(index)
    
            yValues = [] #Corresponding X values
            for index in indices:
                yValues.append(y_transformed[index]) 

            yValues.sort()
            plt.scatter(xValues, yValues, zorder=10)
            
            # Measuring girth as an array output (tuple)
            girth = math.sqrt((xValues[-1] - xValues[0])**2 + (yValues[-1] - yValues[0])**2)
            current_whale_girth.append(girth)
           
            
            currentX = currentX + increment
            
        print("Image ID: ", img_id)
        print("Current Whale's Girth", current_whale_girth)
        girth_measurements.append(current_whale_girth)
        
    # set axis limits
        plt.xlim([minX - 100, maxX + 100])
        plt.ylim([minY - 200, maxY + 200])
        plt.show()
    
    return girth_measurements
    

## Function Calls

In [11]:
detection_results = run_cnn()

Loading weights  /home/clifgray/Code/Mask_RCNN/logs/whale20180613T0247/mask_rcnn_whale_0150.h5


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: res4e_branch2b/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=557661, _device="/job:localhost/replica:0/task:0/device:GPU:0"](res4a_branch2b/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'res4e_branch2b/random_uniform/RandomUniform', defined at:
  File "/home/clifgray/anaconda3/envs/maskrcnn/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/clifgray/anaconda3/envs/maskrcnn/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/clifgray/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_ne

In [ ]:
measure(detection_results) #CNN appends length 

In [ ]:
manual_results = measure(mask_array) # lengths in pixels
#print(manual_results)0cm-

In [ ]:
cnn_lengths = convert_measurements(detection_results)
manual_lengths = convert_measurements(manual_results)

print("CNN Converted Lengths", cnn_lengths)
print("Manual Converted Lengths", manual_lengths)

In [ ]:
print("girth")
cnn_girths = girth(detection_results, cnn_lengths, 10)